In [ ]:
import requests
import time
import os
from IPython.display import display, Markdown

환경 설정 
- Open Router 의 API Key 입력 
- 사용 LLM 모델 입력 

In [ ]:
#from dotenv import load_dotenv
#load_dotenv() 

API_KEY = os.getenv("OPENROUTER_API_KEY")
if not API_KEY:
    raise ValueError("OPENROUTER_API_KEY 환경변수가 설정되어 있지 않습니다.")

MODEL_NAME = "deepseek/deepseek-chat-v3-0324"         # DeepSeek V3 0324
BASE_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

프롬프트 구성
- Agent 별 프롬프트 템플릿

In [ ]:
from datetime import datetime

def get_current_date_string():
    return datetime.today().strftime("%B %Y")  # 예: "May 2025"


def prompt_bullish_independent(economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Bullish Analyst assigned to independently evaluate the investment outlook for {asset_name}.

Assume the current date is {current_date}.  
All analysis must reflect this timeline and economic environment.

Use the provided economic indicators below as your **primary evidence**.  
You may also refer to general financial principles or historical market behavior **only if relevant**—but do not use outdated statistics or price levels.

Provide a clear, structured bullish investment thesis, focusing on macroeconomic trends, valuation support, and upside catalysts.

{economic_data}"""


def prompt_bearish_independent(economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Bearish Analyst assigned to independently assess the risks and downside potential of {asset_name}.

Assume the current date is {current_date}.  
All analysis must reflect this timeline and economic environment.

Use the economic indicators below as your **main foundation**.  
You may incorporate general market knowledge (e.g., rate impact on tech, past downturns) **only when clearly applicable**—but do not reference outdated numbers or irrelevant events.

Provide a structured bearish view focused on macro risks, earnings concerns, and sentiment/technical indicators.

{economic_data}"""


def prompt_bearish_rebuttal(previous_response, economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Bearish Analyst responding to the Bullish Analyst’s assessment of {asset_name}.

Assume the current date is {current_date}.  
All analysis must reflect this timeline and economic environment.

Here is the original bullish opinion:
\"\"\"{previous_response}\"\"\"

Use the same economic data provided below as your **main evidence**.  
You may supplement your reasoning with general market knowledge **only if it helps critique the logic or assumptions made**.

Avoid vague speculation. Present a clear, logical rebuttal focusing on risks and valuation pressure.

{economic_data}"""


def prompt_bullish_rebuttal(previous_response, economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Bullish Analyst responding to the Bearish Analyst’s assessment of {asset_name}.

Assume the current date is {current_date}.  
All analysis must reflect this timeline and economic environment.

Here is the original bearish opinion:
\"\"\"{previous_response}\"\"\"

Use the economic data below as your **primary evidence** to construct a bullish rebuttal.  
You may reference general macro or valuation principles **only to clarify or refine your argument**.

Present a structured counterpoint, addressing key weaknesses in the bearish view.

{economic_data}"""


def prompt_trader(previous_response, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Trader evaluating the investment debate for {asset_name}.

Assume the current date is {current_date}.  
All reasoning must reflect this timeframe.

Below is the full discussion between Bullish and Bearish Analysts:
\"\"\"{previous_response}\"\"\"

Use the provided economic indicators and arguments only—do not introduce new data or assumptions.

Provide a final investment recommendation for {asset_name}.  
You must choose exactly **one** of the following options:  
→ 'Strong Buy', 'Buy', 'Hold', or 'Sell'.

❗️Avoid neutral or ambiguous responses.  
Choose the most appropriate stance based on the **economic signals and debate quality**.

Explain your reasoning in 2–3 clear sentences. Your response will be used in an automated investment decision system."""


def prompt_risk_manager(previous_response, user_profile, current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""You are a Risk Manager tasked with determining position sizing based on the Trader’s recommendation and the user’s investment profile.

Assume the current date is {current_date}.  
Portfolio decisions should reflect current market conditions and macroeconomic risks.

The Trader’s final recommendation is:
\"\"\"{previous_response}\"\"\"

The investor’s profile is as follows:
{user_profile}

Recommend a portfolio allocation (0–100%) for this asset, reflecting the user’s investment horizon, risk appetite, and drawdown tolerance.

Avoid vague suggestions. Provide a specific allocation percentage and justify your sizing decision with reference to both upside opportunity and downside risk.  
Total allocation must not exceed 100%."""


In [ ]:
from datetime import datetime

def get_current_date_string():
    return datetime.today().strftime("%B %Y")  # 예: "May 2025"

def prompt_bullish_independent_ko(economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 {asset_name}에 대한 낙관적인 투자 의견을 독립적으로 제시하는 금융 애널리스트입니다.

현재 날짜는 {current_date}이며, 분석은 반드시 이 시점을 기준으로 이루어져야 합니다.

아래에 제공된 경제 지표를 주요 근거로 사용하세요.  
필요한 경우 일반적인 금융 원칙이나 과거 사례를 보조 논리로 언급할 수 있으나,  
과거 수치나 가격 정보는 사용하지 마세요.  
❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**

거시경제 흐름, 밸류에이션 매력도, 상방 촉매 등을 중심으로  
구조화된 한국어 투자 의견을 작성하세요.

{economic_data}"""

def prompt_bearish_independent_ko(economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 {asset_name}에 대한 비관적인 투자 의견을 독립적으로 제시하는 금융 애널리스트입니다.

현재 날짜는 {current_date}이며, 분석은 반드시 이 시점을 기준으로 이루어져야 합니다.

아래에 제공된 경제 지표를 주요 분석 기반으로 삼고,  
필요 시 일반적인 금융 원칙을 보조적으로 활용할 수 있으나  
구체적인 과거 수치나 시장 데이터는 사용하지 마세요.  
❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**

거시경제 위험, 실적 둔화 가능성, 투자 심리 및 기술적 신호를 중심으로  
구조화된 한국어 투자 리포트를 작성하세요.

{economic_data}"""

def prompt_bearish_rebuttal_ko(previous_response, economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 {asset_name}에 대한 낙관적 분석에 반박하는 금융 애널리스트입니다.

현재 날짜는 {current_date}이며, 분석은 반드시 이 시점을 기준으로 이루어져야 합니다.

다음은 낙관적 관점의 분석입니다:
{previous_response}

아래의 동일한 경제 데이터를 주요 근거로 활용해  
논리적인 반박 의견을 한국어로 제시하세요.  
보조적으로 일반적인 금융 상식을 활용할 수 있으나, 과거 수치는 사용하지 마세요.  
❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**

비관적 관점에서 투자 리스크와 과도한 기대 요소를 지적하고,  
명확하고 구조화된 반론을 제시하세요.

{economic_data}"""

def prompt_bullish_rebuttal_ko(previous_response, economic_data, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 {asset_name}에 대한 비관적 분석에 반박하는 금융 애널리스트입니다.

현재 날짜는 {current_date}이며, 분석은 반드시 이 시점을 기준으로 이루어져야 합니다.

다음은 비관적 관점의 분석입니다:
{previous_response}

아래의 동일한 경제 데이터를 주요 근거로 활용해  
논리적이고 근거 있는 낙관적 반론을 한국어로 작성하세요.  
필요 시 일반적인 시장 원리를 보조 논리로 사용할 수 있습니다.  
❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**

약점 보완, 논리 반박, 추가 근거 중심으로 구조화된 응답을 제공하세요.

{economic_data}"""

def prompt_trader_ko(previous_response, asset_name="QQQ", current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 {asset_name}에 대한 투자 토론을 종합 평가하는 트레이더입니다.

현재 날짜는 {current_date}이며, 판단은 이 시점의 시장 상황을 반영해야 합니다.

다음은 낙관적·비관적 애널리스트 간의 토론 내용입니다:
{previous_response}

주어진 논거와 경제 지표만을 바탕으로  
{asset_name}에 대한 최종 투자 의견을 하나 선택하세요:

→ 'Strong Buy', 'Buy', 'Hold', 또는 'Sell'

❗️중립적이거나 모호한 답변은 피하고,  
**거시경제 신호와 논거의 설득력**을 기준으로 가장 타당한 스탠스를 선택하세요.  
❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**

의견의 근거를 한국어로 2~3문장 이내로 명확히 설명하세요.  
이 응답은 자동화된 투자 판단 시스템에 사용됩니다."""

def prompt_risk_manager_ko(previous_response, user_profile, current_date=None):
    if current_date is None:
        current_date = get_current_date_string()
    return f"""당신은 트레이더의 판단을 바탕으로  
사용자의 투자 성향에 맞는 포트폴리오 비중을 제안하는 리스크 매니저입니다.

현재 날짜는 {current_date}이며, 판단은 최신 시장 환경과 리스크를 반영해야 합니다.

트레이더의 최종 의견:
{previous_response}

사용자의 프로필은 다음과 같습니다:
{user_profile}

자산별 포트폴리오 비중을 0~100% 내에서 하나로 제안하고,  
왜 해당 비중이 적절한지를 리스크 관점에서 설명하세요.  
총합은 100%를 넘지 않아야 하며,  
기대 수익과 하방 리스크를 모두 고려한 판단을 제시하세요.

❗️기술적 지표(RSI, MACD, 이동평균선 등), 재무 약어(CAPAEX, OPEX 등), 기타 지표를 **임의로 생성하거나 추정하지 마세요.**"""


프롬프트 구성 
- Agent 별 프롬트프 템블릿에 들어갈 경제지표

In [ ]:
economic_data_dict  = {
    "QQQ": """"Current Market Price: 520
Interest Rate: 4.5%
CPI: 2.3%
Unemployment Rate:  4.2%
Fear and Greed Index : 69
3-Month Return: -8.7%""",

    "GLD": """Real Interest Rate: 1.2%
USD Index: 104.7
Inflation Expectation: 2.4%
Geopolitical Tension: High
Gold ETF Flow (1M): +$1.2B""",

    "TLT": """10Y Treasury Yield: 4.6%
Fed Funds Rate: 5.25%
Inflation Forecast: 2.2%
GDP Growth Rate: 1.1%
Unemployment: 4.0%""",

    "BTC": """Risk Appetite: Neutral
Institutional Inflows: +$0.9B
ETF Approval Sentiment: Positive
Mining Difficulty: ↑ 4.2%
Market Volatility Index (Crypto): 62""",
}

user_profile = """
Investment Horizon: 3 months
Risk Appetite: Conservative
Maximum Acceptable Drawdown: 10%
"""

LLM API 호출 함수 정의
- temperature 등 파라메터 함께 지정 

In [ ]:
def call_llm(prompt):
    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "top_p": 0.9,
        "frequency_penalty": 0.3,
        "presence_penalty": 0.5,
        "max_tokens": 1000
    }
    res = requests.post(BASE_URL, headers=HEADERS, json=payload)
    time.sleep(1)
    return res.json()["choices"][0]["message"]["content"]

프롬프트 구성
- 자산과 자산에 맞는 경제 지표 선택

In [ ]:
asset_name ="QQQ"
economic_data = economic_data_dict.get(asset_name, "No data available.")

LLM API 호출 시작

In [ ]:
display(Markdown("### 🔵 Bullish Analyst (Round 1)"))
bull1 = call_llm(
    prompt_bullish_independent_ko(
        economic_data=economic_data,
        asset_name=asset_name
    )
)
display(Markdown(bull1))

In [ ]:
display(Markdown("### 🔴 Bearish Analyst (Rebuttal)"))
bear1 = call_llm(
    prompt_bearish_rebuttal_ko(
        previous_response=bull1,
        economic_data=economic_data,
        asset_name=asset_name
    )
)
display(Markdown(bear1))

In [ ]:
display(Markdown("### 🔴 Bearish Analyst (Round2)"))
bear2 = call_llm(
    prompt_bearish_independent_ko(
        economic_data=economic_data,
        asset_name=asset_name
    )
)
display(Markdown(bear2))

In [ ]:
display(Markdown("### 🔵 Bullish Analyst (Rebuttal)"))
bull2 = call_llm(
    prompt_bullish_rebuttal_ko(
        previous_response=bear2,
        economic_data=economic_data,
        asset_name=asset_name
    )
)
display(Markdown(bull2))

In [ ]:
debate_summary = f"[Bullish #1]\n{bull1}\n\n[Bearish #1]\n{bear1}\n\n[Bearish #2]\n{bear2}\n\n[Bullish #2]\n{bull2}"

display(Markdown("### 🟡 Trader Decision"))
trader = call_llm(
    prompt_trader_ko(
        previous_response=debate_summary,
        asset_name=asset_name
    )
)
display(Markdown(trader))

In [ ]:
display(Markdown("### 🟣 Risk Manager Adjustment"))
risk = call_llm(
    prompt_risk_manager_ko(
        previous_response=trader,
        user_profile=user_profile
    )
)
display(Markdown(risk))